# Lateral Movement - Remote Scheduled Task via ATSvc named pipe

#### Author: Rifqi Ardia Ramadhan

ATSvc is a Windows DCE-RPC based protocol used by CIFS hosts (SMB) that is used to access Task Scheduler service across the network. In short, it's a remote Windows task scheduler. On the previous article, _Execution - Scheduled Task from unusual folder_, we can identify scheduled task made, but not how it's made. [CrackMapExec](https://github.com/byt3bl33d3r/CrackMapExec) is one of the powerful tool to do remote execution with this ATSvc, with its  module `atexec`. This article will explain one of the example of the exploitation of ATSvc to get into the Execution of a malicious program.

## Use Case
### Objective
Identify if there's any remote task scheduling that is piped over SMB.

### Log Requirements
Microsoft Windows Security Auditing

### Characteristics
| No | Characteristic | Note | Why|
|----|----------------|------|---|
|  1 | A 5145 EventID is logged| | This event generates every time network share object (file or folder) was accessed. |
|2   | RelativeTargetName (RTN) is `atsvc` || This RTN means that someone requested ATSvc from the network |
|3   | Share Name is \\\\\*\\IPC | | |

### References
https://car.mitre.org/analytics/CAR-2015-04-001/

https://blog.menasec.net/2019/03/threat-hunting-25-scheduled-tasks-for.html

https://github.com/byt3bl33d3r/CrackMapExec

### Sample
Dataset acquired from [EVTX-ATTACK-SAMPLES](https://github.com/sbousseaden/EVTX-ATTACK-SAMPLES)

In [1]:
import evtx
import xml.etree.ElementTree as ET
import json

evtx_file = "logs/Lateral Movement/LM_ScheduledTask_ATSVC_target_host.evtx"  # change the source file to the one you have

parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records())

# all of the above is just to parse the EVTX file as Dictionary

Check if there is EventID 5145 with RTN `atsvc` and ShareName `\\*\IPC$`

In [2]:
for i in parse_json:
    if "<EventID>5145</EventID>" in i['data'] and "\"RelativeTargetName\">atsvc</Data>" in i['data'] and "\"ShareName\">\\\\*\\IPC$</Data>" in i['data']:
        print(i['data'].replace("&quot;", "\""))

<?xml version="1.0" encoding="utf-8"?>
<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">
  <System>
    <Provider Name="Microsoft-Windows-Security-Auditing" Guid="54849625-5478-4994-A5BA-3E3B0328C30D">
    </Provider>
    <EventID>5145</EventID>
    <Version>0</Version>
    <Level>0</Level>
    <Task>12811</Task>
    <Opcode>0</Opcode>
    <Keywords>0x8020000000000000</Keywords>
    <TimeCreated SystemTime="2019-03-19 00:02:04.241919 UTC">
    </TimeCreated>
    <EventRecordID>566832</EventRecordID>
    <Correlation>
    </Correlation>
    <Execution ProcessID="452" ThreadID="460">
    </Execution>
    <Channel>Security</Channel>
    <Computer>WIN-77LTAPHIQ1R.example.corp</Computer>
    <Security>
    </Security>
  </System>
  <EventData>
    <Data Name="SubjectUserSid">S-1-5-21-1587066498-1489273250-1035260531-500</Data>
    <Data Name="SubjectUserName">Administrator</Data>
    <Data Name="SubjectDomainName">EXAMPLE</Data>
    <Data Name="SubjectLogonId">0x17e2c0</